![](https://storage.googleapis.com/mle-courses-prod/users/61b869ca9c3c5e00292bb42d/private-files/c97b9af0-796a-11ef-80c6-c1c8552cee67-Screen_Shot_2024_09_23_at_12.14.59.png)

In [3]:
# Document loading, retrieval methods and text splitting
%pip install -qU langchain langchain_community

# Local vector store via Chroma
%pip install -qU langchain_chroma

# Local inference and embeddings via Ollama
%pip install -qU langchain_ollama

In [4]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Installing ollama to /usr/local
100 13320    0 13320    0     0  56338      0 --:--:-- --:--:-- --:--:-- 56440
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [5]:
import subprocess
import time
import threading

# Start the ollama server in a new process
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Function to print server output
def print_output(process):
    while True:
        output = process.stdout.readline()
        if output == b'' and process.poll() is not None:
            break
        if output:
            print(output.strip().decode('utf-8'))
        time.sleep(1)

# Start a thread to print server output
thread = threading.Thread(target=print_output, args=(process,))
thread.start()

print("Ollama server is running in the background")


Ollama server is running in the background


In [6]:
!ollama pull llama3.1:8b

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is:

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIDwy9Fc0V6yQ5QRFbdSWnQeEiI3Jc9jqZaSm/v8EdUVq

[GIN] 2024/09/24 - 08:15:15 | 200 |      59.161µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏ 485 KB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   1% ▕▏  23 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   1% ▕▏  56 MB/4.7 GB                  pulling mani

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain.chains import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel
import uuid

# Initialize the Ollama LLaMA 3 model
class AgenticChunkerLLaMA3:
    def __init__(self):
        # Start Ollama server
        self.llama_model = ChatOllama(model="llama3.1:8b")

        self.chunks = {}
        self.id_truncate_limit = 5
        self.generate_new_metadata_ind = True
        self.print_logging = True

    def add_propositions(self, propositions):
        for proposition in propositions:
            self.add_proposition(proposition)


    def add_proposition_to_chunk(self, chunk_id, proposition):
        # Add the new proposition to the existing chunk
        self.chunks[chunk_id]['propositions'].append(proposition)

        # Update the chunk's summary and title if needed
        if self.generate_new_metadata_ind:
            self.chunks[chunk_id]['summary'] = self._update_chunk_summary(self.chunks[chunk_id])
            self.chunks[chunk_id]['title'] = self._update_chunk_title(self.chunks[chunk_id])

    def _update_chunk_summary(self, chunk):
        prompt = f"""
        A new sentence has been added to this chunk. Update the summary to reflect the current content.

        Chunk sentences:
        {', '.join(chunk['propositions'])}

        Current summary: {chunk['summary']}

        Provide the updated summary.
        """

        response = self.llama_model.invoke(prompt).content.strip()
        return response

    def _update_chunk_title(self, chunk):
        prompt = f"""
        Based on this updated chunk summary: '{chunk['summary']}', provide a concise new title.
        """

        response = self.llama_model.invoke(prompt).content.strip()
        return response


    def add_proposition(self, proposition):
        if self.print_logging:
            print(f"\nAdding: '{proposition}'")

        if len(self.chunks) == 0:
            if self.print_logging:
                print("No chunks, creating a new one")
            self._create_new_chunk(proposition)
            return

        chunk_id = self._find_relevant_chunk(proposition)

        print('---chunk_id', chunk_id)

        if chunk_id:
            if self.print_logging:
                print(f"Chunk Found ({self.chunks[chunk_id]['chunk_id']}), adding to: {self.chunks[chunk_id]['title']}")
            self.add_proposition_to_chunk(chunk_id, proposition)
            return
        else:
            if self.print_logging:
                print("No chunks found")
            self._create_new_chunk(proposition)

    def _create_new_chunk(self, proposition):
        new_chunk_id = str(uuid.uuid4())[:self.id_truncate_limit]
        new_chunk_summary = self._get_new_chunk_summary(proposition)
        new_chunk_title = self._get_new_chunk_title(new_chunk_summary)

        self.chunks[new_chunk_id] = {
            'chunk_id': new_chunk_id,
            'propositions': [proposition],
            'title': new_chunk_title,
            'summary': new_chunk_summary,
            'chunk_index': len(self.chunks)
        }
        if self.print_logging:
            print(f"Created new chunk ({new_chunk_id}): {new_chunk_title}")

    def _get_new_chunk_summary(self, proposition):
        prompt = f"""
        You are managing chunks representing groups of sentences with a similar topic.
        Please summarize this new proposition in one sentence:

        Proposition: {proposition}
        """

        response = self.llama_model.invoke(prompt).content
        return response.strip()

    def _get_new_chunk_title(self, summary):
        prompt = f"""
        Given this summary: '{summary}', provide a concise title generalizing the chunk's content.
        """

        response = self.llama_model.invoke(prompt).content
        return response.strip()

    def _find_relevant_chunk(self, proposition):
        current_chunk_outline = self.get_chunk_outline()
        prompt = f"""
        Determine if the following sentence belongs to any existing chunk.

        Sentence: '{proposition}'

        Current chunks:
        {current_chunk_outline}

        If it belongs, provide only the chunk ID. If it doesn't belong to any, respond with 'No chunks'.
        """

        response = self.llama_model.invoke(prompt).content.strip()

        # Extract only the chunk ID
        if "No chunks" in response:
            return None
        else:
            # Use regex to find the chunk ID in the response
            import re
            match = re.search(r'\b\w{5}\b', response)  # Assuming the chunk ID is always 5 alphanumeric characters
            if match:
                return match.group(0)
            return None


    def get_chunk_outline(self):
        chunk_outline = ""
        for chunk_id, chunk in self.chunks.items():
            chunk_outline += f"Chunk ID: {chunk['chunk_id']}\nChunk Title: {chunk['title']}\nSummary: {chunk['summary']}\n\n"
        return chunk_outline

    def pretty_print_chunks(self):
        print(f"\nYou have {len(self.chunks)} chunks\n")
        for chunk_id, chunk in self.chunks.items():
            print(f"Chunk #{chunk['chunk_index']}")
            print(f"Chunk ID: {chunk_id}")
            print(f"Summary: {chunk['summary']}")
            print("Propositions:")
            for prop in chunk['propositions']:
                print(f"    - {prop}")
            print("\n\n")


# Instantiate your chunker using LLaMA 3
llama_chunker = AgenticChunkerLLaMA3()

# Sample propositions
propositions = [
    "The only way to be successful is to get up at the crack of dawn – or so the story goes.",
    "But early rising productivity is not a one-size-fits-all situation, Bryan Lufkin finds.",
    "Being successful means waking up early – or so we’re constantly told."
]

llama_chunker.add_propositions(propositions)
llama_chunker.pretty_print_chunks()


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)



Adding: 'The only way to be successful is to get up at the crack of dawn – or so the story goes.'
No chunks, creating a new one
INFO [main] build info | build=10 commit="9225b05" tid="138105252204544" timestamp=1727165781
INFO [main] system info | n_threads=1 n_threads_batch=1 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="138105252204544" timestamp=1727165781 total_threads=2
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="6" port="32991" tid="138105252204544" timestamp=1727165781
INFO [main] model loaded | tid="138105252204544" timestamp=1727165793
[GIN] 2024/09/24 - 08:16:34 | 200 | 14.236513464s |       127.0.0.1 | POST     "/api/chat"
Created new chunk (74514): "Uncertainty Surrounding Early Rising and Success"

Adding: 